## Step 1: Ingest Data

We will use RESTful API to retrieve traffic camera data from the public City of Calgary website https://data.calgary.ca/Transportation-Transit/Traffic-Cameras/k7p9-kppz/about_data

We will use the python library to help make our requests.

In [2]:
!python -V

Python 3.10.12


In [ ]:
#request
import requests
import pandas as pd
import io

api_url = 'https://data.calgary.ca/resource/k7p9-kppz.csv'


response = requests.get(api_url)

if response.status_code == 200:
  data = io.StringIO(response.text)
  df = pd.read_csv(data)
  display(df.head())
else:
  print("Failed to get data")

,camera_url,quadrant,camera_location,point
0,Camera 158 (http://trafficcam.calgary.ca/loc15...,SW,26 Ave / 4 St SW,POINT (-114.0716774 51.0296112)
1,Camera 30 (http://trafficcam.calgary.ca/loc29....,SE,114 Avenue / 52 Street SE,POINT (-113.958331 50.9504395)
2,Camera 142 (http://trafficcam.calgary.ca/loc14...,SW,90 Avenue / 24 Street SW,POINT (-114.1177502 50.9724213)
3,Camera 90 (http://trafficcam.calgary.ca/loc89....,NW/NE,Samis Road / Centre Street North,POINT (-114.0624739 51.0549834)
4,Camera 38 (https://trafficcam.calgary.ca/loc37...,NW,Crowchild Trail / Shaganappi Trail NW,POINT (-114.149379 51.0988494)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   camera_url       176 non-null    object
 1   quadrant         176 non-null    object
 2   camera_location  176 non-null    object
 3   point            176 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB


In [ ]:
#transforming dataframe
df[['Camera_Number', 'URL']] = df['camera_url'].str.extract(r'Camera (\d+) \((http[^\)]+)\)')
df[['longitude', 'latitude']] = df['point'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'])
df['longitude'] = pd.to_numeric(df['longitude'])
df.drop(columns=['camera_url','point'], inplace = True)
df['Camera_Number'] = df['Camera_Number'].astype(int)
display(df.head())

,quadrant,camera_location,Camera_Number,URL,longitude,latitude
0,SW,26 Ave / 4 St SW,158,http://trafficcam.calgary.ca/loc157.jpg,-114.071677,51.029611
1,SE,114 Avenue / 52 Street SE,30,http://trafficcam.calgary.ca/loc29.jpg,-113.958331,50.950440
2,SW,90 Avenue / 24 Street SW,142,http://trafficcam.calgary.ca/loc141.jpg,-114.117750,50.972421
3,NW/NE,Samis Road / Centre Street North,90,http://trafficcam.calgary.ca/loc89.jpg,-114.062474,51.054983
4,NW,Crowchild Trail / Shaganappi Trail NW,38,https://trafficcam.calgary.ca/loc37.jpg,-114.149379,51.098849


In [ ]:
from IPython.display import Image, display


image_url = 'http://trafficcam.calgary.ca/loc29.jpg'

display(Image(url=image_url))


In [ ]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab


In [ ]:
#cloning git and installing yolov5
import sys
!rm -rf /content/yolov5 #removing the existing directory
!git clone https://github.com/ultralytics/yolov5 --tag 6.2  # clone
!mv /content/6.2 /content/yolov5
%pip install -qr /content/yolov5/requirements.txt  # install
sys.path.insert(0,'/content/yolov5')

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.6/107.7 GB disk)


In [ ]:
import urllib.request
import shutil
import os
from IPython.display import Image, display




images_dir = "/content/images/"
yolo_output_dir = "/content/yolov5/runs/detect/"

os.makedirs(images_dir, exist_ok=True)
os.makedirs(yolo_output_dir, exist_ok=True)

!rm -rf {yolo_output_dir}*

failed_downloads = []

for i, row in df.iterrows():
    image_url = row['URL']
    image_name = f"image_{i}.jpg"
    local_img_file = os.path.join(images_dir, image_name)

    try:

        with urllib.request.urlopen(image_url) as response, open(local_img_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
    except urllib.error.HTTPError as e:
        print(f"Failed to download {image_url}: {e.reason}")
        failed_downloads.append((image_url, e.code))
        continue


# run yolo
!python /content/yolov5/detect.py --weights yolov5s.pt --img 1024 \
  --conf 0.25 --source {images_dir} --name "batch"

output_images_dir = os.path.join(yolo_output_dir, "images")
for image_file in os.listdir(output_images_dir):
    display(Image(filename=os.path.join(output_images_dir, image_file)))

print("Failed downloads:", failed_downloads)




Failed to download http://trafficcam.calgary.ca/loc22.jpg: Not Found
detect: weights=['yolov5s.pt'], source=/content/images/, data=yolov5/data/coco128.yaml, imgsz=[1024, 1024], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=batch, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CPU

100% 14.1M/14.1M [00:00<00:00, 38.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/175 /content/images/image_0.jpg: 768x1024 2 traffic lights, 1049.2ms
image 2/175 /content/images/image_1.jpg: 768x1024 1 person, 2 traffic lights, 907.4ms
image 3/175 /content/images/image_10.jpg: 768x1024 5 traffic lights, 892.8ms
image

FileNotFoundError: [Errno 2] No such file or directory: '/content/yolov5/runs/detect/images'

Sets up the directories for storing images and YOLOv5 outputs.
Clears the YOLOv5 detect output directory to avoid clutter from previous runs.
Downloads the specified image from your DataFrame and saves it locally.
Runs the YOLOv5 model on the saved image to perform object detection.
Finds the latest output directory created by YOLOv5 (since YOLOv5 creates a new directory for each run) and constructs the path to the detected image.
Displays the image with detected objects using IPython's Image display function.

In [ ]:
import torch
import os
import pandas as pd

# Load the YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


images_dir = "/content/images/"

all_detections = []


for image_name in os.listdir(images_dir):
    local_img_file = os.path.join(images_dir, image_name)
    results = yolo_model(local_img_file)
    dt = results.pandas().xyxy[0]

    detection_counts = dt['name'].value_counts().reset_index()
    detection_counts.columns = ['name', 'count']

    detections_pivot = detection_counts.pivot(index=None, columns='name', values='count').fillna(0).astype(int)


    if detections_pivot.empty:
        detections_pivot = pd.DataFrame([{}])

    camera_number = int(image_name.split('_')[-1].split('.')[0]) + 1
    detections_pivot['Camera Number'] = camera_number

    all_detections.append(detections_pivot)

all_detections_df = pd.concat(all_detections, ignore_index=True)

all_detections_df = all_detections_df.fillna(0)

cols = list(all_detections_df)
cols.insert(0, cols.pop(cols.index('Camera Number')))
all_detections_df = all_detections_df.loc[:, cols]

consolidated_detections_df = all_detections_df.groupby('Camera Number').sum().reset_index()










Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
df_detections= consolidated_detections_df.filter(items=['Camera Number','car', 'truck','bus'])
final_detection_table = pd.merge(df_detections, df, how='inner', left_on ='Camera Number', right_on='Camera_Number')
final_detection_table.head(30)

,Camera Number,car,truck,bus,quadrant,camera_location,Camera_Number,URL,longitude,latitude
0,1,0.0,0.0,0.0,SW/SE,Glenmore Trail / Macleod Trail S,1,http://trafficcam.calgary.ca/loc0.jpg,-114.071585,50.993989
1,2,0.0,0.0,0.0,SE,Peigan Trail / Barlow Trail SE,2,http://trafficcam.calgary.ca/loc1.jpg,-114.002689,51.014972
2,3,2.0,0.0,0.0,SE,50 Avenue / Barlow Trail SE,3,http://trafficcam.calgary.ca/loc2.jpg,-114.001845,51.008623
3,4,2.0,0.0,0.0,NE,Memorial Drive / 52 Street E,4,http://trafficcam.calgary.ca/loc3.jpg,-113.955818,51.053253
4,5,0.0,0.0,0.0,NW,Kensington Road / Crowchild Trail NW,5,http://trafficcam.calgary.ca/loc4.jpg,-114.118088,51.052487
5,6,1.0,0.0,0.0,SW,4 Avenue / 9 Street SW,6,http://trafficcam.calgary.ca/loc5.jpg,-114.083522,51.049960
6,7,0.0,0.0,0.0,SW,Glenmore Trail / Elbow Drive SW (East),7,http://trafficcam.calgary.ca/loc6.jpg,-114.084236,50.993916
7,8,4.0,0.0,0.0,SE,Stoney Trail / 52 Street SE,8,http://trafficcam.calgary.ca/loc7.jpg,-113.953747,50.904018
8,9,2.0,0.0,0.0,SW/SE,Heritage Drive / Macleod Trail S,9,http://trafficcam.calgary.ca/loc8.jpg,-114.071775,50.979387
9,10,1.0,0.0,1.0,SW/SE,Bannister Road / Macleod Trail S,10,http://trafficcam.calgary.ca/loc9.jpg,-114.071651,50.925155


In [ ]:
timeapi_url = 'https://www.timeapi.io/api/Time/current/zone?timeZone=America/Edmonton'

response = requests.get(timeapi_url)
time_data = response.json()

current_date = time_data['date']
current_time = time_data['time']
current_dayofweek = time_data['dayOfWeek']
df_time =pd.DataFrame({
    'Current Date':[current_date],
    'Current Time':[current_time],
    'Day of Week' :[current_dayofweek]
})

final_detection_table = final_detection_table.assign(
        **{'Current Date': current_date, 'Current Time': current_time, 'Day of Week': current_dayofweek}
    )

final_detection_table


,Camera Number,car,truck,bus,quadrant,...,longitude,latitude,Current Date,Current Time,Day of Week
0,1,0.0,0.0,0.0,SW/SE,...,-114.071585,50.993989,03/19/2024,23:39,Tuesday
1,2,0.0,0.0,0.0,SE,...,-114.002689,51.014972,03/19/2024,23:39,Tuesday
2,3,2.0,0.0,0.0,SE,...,-114.001845,51.008623,03/19/2024,23:39,Tuesday
3,4,2.0,0.0,0.0,NE,...,-113.955818,51.053253,03/19/2024,23:39,Tuesday
4,5,0.0,0.0,0.0,NW,...,-114.118088,51.052487,03/19/2024,23:39,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...
169,172,0.0,0.0,0.0,NE,...,-113.919492,51.154137,03/19/2024,23:39,Tuesday
170,173,2.0,0.0,0.0,NE,...,-114.035252,51.142317,03/19/2024,23:39,Tuesday
171,174,0.0,0.0,0.0,SW,...,-114.211172,51.066886,03/19/2024,23:39,Tuesday
172,175,0.0,0.0,0.0,SW,...,-114.210956,51.038191,03/19/2024,23:39,Tuesday


In [ ]:
final_detection_table.to_csv('Traffic_detection.csv', index=False)